NAIVE BAYES

In [ ]:
# PCA 
def standardize_data(X):
    mean = np.mean(X, axis=0)
    std_dev = np.std(X, axis=0)
    X_scaled = (X - mean) / std_dev
    return X_scaled, mean, std_dev

X_train_scaled, mean_train, std_dev_train = standardize_data(x_train)
X_test_scaled = (x_test - mean_train) / std_dev_train

cov_matrix = np.cov(X_train_scaled.T)

eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)

k = 5 
top_indices = np.argsort(eigenvalues)[::-1][:k]
top_eigenvectors = eigenvectors[:, top_indices]

X_train_pca = X_train_scaled.dot(top_eigenvectors)
X_test_pca = X_test_scaled.dot(top_eigenvectors)

naive_bayes = GaussianNB()
naive_bayes.fit(X_train_pca, y_train)

y_pred = naive_bayes.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy in naive bayes:", accuracy)


DECISION TREE

In [ ]:
#decision tree

k = 10 
accuracy_scores = []
fold_size = len(X_train_pca) // k
for i in range(k):
    start_idx = i * fold_size
    end_idx = start_idx + fold_size
    X_val_fold = X_train_pca[start_idx:end_idx]
    y_val_fold = y_train[start_idx:end_idx]
    
    X_train_fold = np.concatenate([X_train_pca[:start_idx], X_train_pca[end_idx:]], axis=0)
    y_train_fold = np.concatenate([y_train[:start_idx], y_train[end_idx:]], axis=0)
    
    decision_tree = DecisionTreeClassifier()
    decision_tree.fit(X_train_fold, y_train_fold)

    y_pred_fold = decision_tree.predict(X_val_fold)

    accuracy_fold = accuracy_score(y_val_fold, y_pred_fold)
    accuracy_scores.append(accuracy_fold)

average_accuracy = np.mean(accuracy_scores)
print("Average Accuracy:", average_accuracy)


RANDOM FOREST

In [ ]:
# Random Forest
random_forest = RandomForestClassifier()
random_forest.fit(X_train_pca, y_train)
rf_y_pred = random_forest.predict(X_test_pca)
rf_accuracy = accuracy_score(y_test, rf_y_pred)
print("Random Forest Accuracy:", rf_accuracy)
# Gradient Boosting
gradient_boosting = GradientBoostingClassifier()
gradient_boosting.fit(X_train_pca, y_train)
gb_y_pred = gradient_boosting.predict(X_test_pca)
gb_accuracy = accuracy_score(y_test, gb_y_pred)
print("Gradient Boosting Accuracy:", gb_accuracy)

BOOSTING TECHNIQUES

In [ ]:

gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

gb_grid_search = GridSearchCV(GradientBoostingClassifier(), param_grid=gb_param_grid, cv=5)
gb_grid_search.fit(x_train, y_train)

best_gb_model = gb_grid_search.best_estimator_

dump(best_gb_model, 'best_gradient_boosting_model.joblib')

gb_y_pred = best_gb_model.predict(x_train)
gb_accuracy = accuracy_score(y_train, gb_y_pred)
print("Gradient Boosting Accuracy using GridSearchCV:", gb_accuracy)

PREDICTION

In [ ]:
best_gb_model = load('best_gradient_boosting_model.joblib')
new_data = pd.read_csv(r'C:\Users\Padmajaa\OneDrive - SSN Trust\4th sem\ML lab\PROJECT\flask\profile_info.csv')  # Replace 'your_data_for_prediction.csv' with the path to your CSV file

X_new = new_data  
predictions = best_gb_model.predict(X_new)
print("Predictions:")
for pred in predictions:
    print("Fake" if pred == 1 else "Not Fake")